In [1]:
import os
#go back one directory
os.chdir("..")
# now go into source directory
os.chdir("source")


In [ ]:
from tools.spaceX_ephem_tools import spacex_ephem_to_dataframe, parse_spacex_datetime_stamps, read_spacex_ephemeris, spacex_ephem_to_dataframe
from tools.utilities import yyyy_mm_dd_hh_mm_ss_to_jd

In [ ]:
# read starlink ephem
# plot altitude time series
# write covariance getter function
# animated covariance plot
# read positions and velocities into orekit.measurement 

In [4]:
_start,  = read_spacex_ephemeris("/Users/charlesc/Documents/GitHub/ERP_tools/external/ephems/starlink/MEME_57632_STARLINK-30309_3530645_Operational_1387262760_UNCLASSIFIED.txt")

In [6]:
spacex_eph_df = spacex_ephem_to_dataframe("/Users/charlesc/Documents/GitHub/ERP_tools/external/ephems/starlink/MEME_57632_STARLINK-30309_3530645_Operational_1387262760_UNCLASSIFIED.txt")

In [7]:
spacex_eph_df.head()

,jd_time,x,y,z,u,v,w
0,2.460298e+06,3163.377902,4612.858121,-4107.476805,-6.736341,2.336685,-2.565039
1,2.460298e+06,2752.704348,4743.069814,-4252.438989,-6.947869,2.002163,-2.265308
2,2.460298e+06,2330.230006,4852.947224,-4379.120144,-7.129576,1.659119,-1.955894
3,2.460298e+06,1897.767239,4942.022916,-4486.979068,-7.280695,1.309018,-1.638123
4,2.460298e+06,1457.171089,5009.918162,-4575.554948,-7.400593,0.953355,-1.313355


In [8]:
import pandas as pd

In [13]:
def spacex_ephem_to_df_w_cov(ephem_path: str) -> pd.DataFrame:
    """
    Convert SpaceX ephemeris data, including covariance terms, into a pandas DataFrame.

    Parameters
    ----------
    ephem_path : str
        Path to the ephemeris file.

    Returns
    -------
    pd.DataFrame
        DataFrame containing parsed SpaceX ephemeris data, including covariance terms.
    """
    with open(ephem_path) as f:
        lines = f.readlines()

    # Remove header lines and select every 4th line starting from the first data line
    t_xyz_uvw = lines[4::4]

    # Extract t, x, y, z, u, v, w
    t = [float(i.split()[0]) for i in t_xyz_uvw]
    x = [float(i.split()[1]) for i in t_xyz_uvw]
    y = [float(i.split()[2]) for i in t_xyz_uvw]
    z = [float(i.split()[3]) for i in t_xyz_uvw]
    u = [float(i.split()[4]) for i in t_xyz_uvw]
    v = [float(i.split()[5]) for i in t_xyz_uvw]
    w = [float(i.split()[6]) for i in t_xyz_uvw]

    # Extract the 21 covariance terms (3 lines after each primary data line)
    covariance_data = {f'cov_{i+1}': [] for i in range(21)}
    for i in range(5, len(lines), 4):  # Start from the first covariance line
        cov_lines = lines[i:i+3]  # Get the three lines of covariance terms
        cov_values = ' '.join(cov_lines).split()
        for j, value in enumerate(cov_values):
            covariance_data[f'cov_{j+1}'].append(float(value))

    # Construct the DataFrame
    spacex_ephem_df = pd.DataFrame({
        't': t, 'x': x, 'y': y, 'z': z, 'u': u, 'v': v, 'w': w,
        **covariance_data
    })

    return spacex_ephem_df


In [14]:
spacex_ephem_dfwcov = spacex_ephem_to_df_w_cov("/Users/charlesc/Documents/GitHub/ERP_tools/external/ephems/starlink/MEME_57632_STARLINK-30309_3530645_Operational_1387262760_UNCLASSIFIED.txt")

In [15]:
spacex_ephem_dfwcov.columns

Index(['t', 'x', 'y', 'z', 'u', 'v', 'w', 'cov_1', 'cov_2', 'cov_3', 'cov_4',
       'cov_5', 'cov_6', 'cov_7', 'cov_8', 'cov_9', 'cov_10', 'cov_11',
       'cov_12', 'cov_13', 'cov_14', 'cov_15', 'cov_16', 'cov_17', 'cov_18',
       'cov_19', 'cov_20', 'cov_21'],
      dtype='object')

In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from matplotlib.colors import SymLogNorm

# Create the output directory if it doesn't exist
output_dir = "output/covariance_heatmaps/starlink"
os.makedirs(output_dir, exist_ok=True)

# Function to create a symmetric correlation matrix from lower triangular covariance elements
def create_symmetric_corr_matrix(lower_triangular_data):
    # Create the symmetric covariance matrix
    cov_matrix = np.zeros((6, 6))
    row, col = np.tril_indices(6)
    cov_matrix[row, col] = lower_triangular_data
    cov_matrix = cov_matrix + cov_matrix.T - np.diag(cov_matrix.diagonal())

    # Convert to correlation matrix
    std_dev = np.sqrt(np.diag(cov_matrix))
    corr_matrix = np.divide(cov_matrix, std_dev[:, None])
    corr_matrix = np.divide(corr_matrix, std_dev[None, :])
    np.fill_diagonal(corr_matrix, 1)  # Fill diagonal with 1s for self-correlation
    return corr_matrix

# Axis labels
axis_labels = ['x', 'y', 'z', 'u', 'v', 'w']
# Assuming spacex_ephem_dfwcov is your DataFrame and it contains columns 'cov_1' to 'cov_21'
all_cov_data = spacex_ephem_dfwcov.loc[:, 'cov_1':'cov_21'].values

# Iterate through the first 300 items
filenames = []
for i in range(min(300, len(spacex_ephem_dfwcov))):
    # Create the symmetric correlation matrix
    corr_matrix = create_symmetric_corr_matrix(all_cov_data[i, :])

    # Create a heatmap
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(corr_matrix, annot=True, cmap='viridis', xticklabels=axis_labels, yticklabels=axis_labels, vmin=-1, vmax=1)
    plt.title(f'Starlink-57632 Correlation: {(spacex_ephem_dfwcov.loc[i, "t"])}')

    # Save the heatmap to a file
    filename = f'{output_dir}/spacex_corr_hm{i}.png'
    plt.savefig(filename, dpi=120)
    filenames.append(filename)
    plt.close()

# Now you can use the filenames list to create a GIF as previously described


In [37]:
import imageio

def create_animation(filenames, animation_path):
    images = [imageio.imread(filename) for filename in filenames]
    imageio.mimsave(animation_path, images, duration=0.5)

# Path for the output GIF
animation_path = "output/covariance_heatmaps/starlink/spacex_corr_heatmaps.gif"

# Create the animation
create_animation(filenames, animation_path)

/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_57334/4269234183.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(filename) for filename in filenames]
